In [1]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader 
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain, ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory, ConversationBufferMemory,ConversationSummaryMemory

from langchain_community.llms import HuggingFaceEndpoint





In [2]:
import json

# Öffne die JSON-Datei und lade den Inhalt
with open('/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/api_token.json', 'r') as api_file:
    api_token_file = json.load(api_file)

# Extrahiere die Variable aus den Daten
api_token = api_token_file['Hugging_face_token']

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50, 
    length_function = len)
text_splitter

In [5]:
filepath = '/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/doc_1.pdf'
loader = PyPDFLoader(filepath)
chunks = loader.load_and_split(text_splitter=text_splitter)

In [6]:
for chunk in chunks[4:5]:
    print("Page content: \n", chunk.page_content),
    print("Page_metadata: \n", chunk.metadata),
    print("----------------------------")

Page content: 
 significant improvements in computational efficiency through factorization tricks [ 21] and conditional
computation [ 32], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduc-
tion models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [ 2,19]. In all but a few cases [ 27], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100

In [8]:
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-large-zh-v1.5")

config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<00:00, 532kB/s]
README.md: 100%|██████████| 30.3k/30.3k [00:00<00:00, 36.3MB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 189kB/s]
config.json: 100%|██████████| 1.00k/1.00k [00:00<00:00, 8.36MB/s]
model.safetensors: 100%|██████████| 1.30G/1.30G [03:35<00:00, 6.04MB/s]
tokenizer_config.json: 100%|██████████| 394/394 [00:00<00:00, 2.46MB/s]
vocab.txt: 100%|██████████| 110k/110k [00:00<00:00, 575kB/s]
tokenizer.json: 100%|██████████| 439k/439k [00:00<00:00, 1.46MB/s]
1_Pooling/config.json: 100%|██████████| 191/191 [00:00<00:00, 899kB/s]


In [9]:
embedding = embedding_function.embed_documents("This is a test sentence.")

print(embedding[0])
print("Dimension of Embedding: ", len(embedding[0]))

[0.030692150816321373, -0.031502220779657364, -0.017730996012687683, 0.035333432257175446, -0.028013229370117188, 0.04366171732544899, -0.017326941713690758, -0.02879144996404648, -0.018377207219600677, -0.03722976893186569, -0.006641303189098835, 0.03447132557630539, -0.022506963461637497, -0.0067087761126458645, -0.00989031232893467, -0.023188108578324318, -0.003284014528617263, 0.024761414155364037, 0.014999981969594955, -0.01783912256360054, -0.023429516702890396, 0.008491289801895618, -0.041361816227436066, 0.001655183150433004, 0.026237130165100098, -0.032837141305208206, -0.01661311462521553, 0.03826865926384926, -0.026053553447127342, -0.025660574436187744, 0.00018425437156111002, 0.057974353432655334, 0.012534989975392818, -0.026392772793769836, -0.021217787638306618, -0.004844049923121929, 0.016133053228259087, -0.007580542471259832, 0.028098614886403084, 0.0007906158571131527, 0.02106611058115959, -0.00990001205354929, 0.029389623552560806, -0.0029493358451873064, 0.00995842

In [10]:
db = Chroma.from_documents(chunks, embedding_function)

In [11]:
print("Chunks in DB:", db._collection.count())

Chunks in DB: 48


In [12]:
query = "Write a summary of the first page of the document."
retriever = db.as_retriever()
retriever.get_relevant_documents(query)

[Document(page_content='recurrent layers, by a factor of k. Separable convolutions [ 6], however, decrease the complexity\nconsiderably, to O(k·n·d+n·d2). Even with k=n, however, the complexity of a separable\nconvolution is equal to the combination of a self-attention layer and a point-wise feed-forward layer,\nthe approach we take in our model.\nAs side benefit, self-attention could yield more interpretable models. We inspect attention distributions\nfrom our models and present and discuss examples in the appendix. Not only do individual attention\nheads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic\nand semantic structure of the sentences.\n5 Training\nThis section describes the training regime for our models.\n5.1 Training Data and Batching\nWe trained on the standard WMT 2014 English-German dataset consisting of about 4.5 million\nsentence pairs. Sentences were encoded using byte-pair encoding [ 3], which has a shared source-', 

In [13]:
llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', 
                     huggingfacehub_api_token=api_token,  
                     model_kwargs={"max_length": 300})


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/riccardo/.cache/huggingface/token
Login successful


In [14]:
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    chain_type = "stuff",  
    retriever = retriever
    )

qa_with_sources

RetrievalQAWithSourcesChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a w

In [23]:
query = "Write a summary of the key findings of the document."
qa_with_sources.invoke(query)

{'question': 'Write a summary of the key findings of the document.',
 'answer': ' The document summarizes research on a neural machine translation model, specifically a long short-term memory (LSTM) network, that uses self-attention mechanisms to allow every position in the decoder to attend over all positions in the input sequence, while preventing leftward information flow to preserve the auto-regressive property. The researchers also discuss the benefits of using separable convolutions to decrease complexity and provide more interpretable models. The training was conducted on the WMT 2014 English-German dataset using byte-pair encoding.\n',
 'sources': ''}